In [83]:
import pandas as pd
import joblib

df = pd.read_csv("호선정리.csv", encoding='CP949')
list_data = df.values.tolist()

def get_station_number(station_name, station_data):
    for item in station_data:
        if item[2] == station_name:
            return int(item[0])
    return None

def determine_direction(start_station, end_station, station_data):
    start_num = get_station_number(start_station, station_data)
    end_num = get_station_number(end_station, station_data)
    return "상선" if (end_num - start_num) > 0 else "하선"

def compute_average_congestion(congestion_values):
    return sum(congestion_values) / len(congestion_values)

def print_congestion_predictions(start_station, end_station, input_YEAR, input_MONTH, input_DAY, input_LINE, input_TIME, station_data):
    filtered_station_data = [item for item in station_data if item[1] == input_LINE]
    stations = [item[2] for item in filtered_station_data]
    
    start_idx = stations.index(start_station)
    end_idx = stations.index(end_station)
    if start_idx > end_idx:
        start_idx, end_idx = end_idx, start_idx

    direction = determine_direction(start_station, end_station, station_data)
    
    model = joblib.load(f'XGB_reg_num{input_LINE}.pkl')

    congestion_values = []
    for idx in range(start_idx, end_idx + 1):
        station = stations[idx]
        input_data = pd.DataFrame({
            'YEAR': [input_YEAR],
            'MONTH': [input_MONTH],
            'DAY': [input_DAY],
            'STATION': [station],
            'DIRECTION': [direction],
            'TIME_00': [input_TIME]
        })
        
        prediction = model.predict(input_data)
        print(f"역: {station}, 예측 혼잡도: {prediction[0]:.2f}")
        congestion_values.append(prediction[0])

    avg_congestion = compute_average_congestion(congestion_values)
    print(f"평균 혼잡도: {avg_congestion:.2f}")

In [85]:
print_congestion_predictions('서울역', '시청', 2023, 5, '월', 1, 'TIME_18', list_data)

역: 서울역, 예측 혼잡도: 49.66
역: 시청, 예측 혼잡도: 65.14
평균 혼잡도: 57.40


In [81]:
import pandas as pd
import joblib

df = pd.read_csv("호선정리.csv", encoding='CP949')
list_data = df.values.tolist()

def get_station_number(station_name, station_data):
    for item in station_data:
        if item[2] == station_name:
            return int(item[0])
    return None

def determine_direction_and_indices(start_station, end_station, stations, line_num):
    start_idx = stations.index(start_station)
    end_idx = stations.index(end_station)
    
    if line_num == 2:
        if abs(end_idx - start_idx) > len(stations) // 2:
            if start_idx < end_idx:
                direction = "하선"
                start_idx, end_idx = end_idx, start_idx + len(stations)
            else:
                direction = "상선"
                end_idx += len(stations)
        else:
            direction = "상선" if start_idx < end_idx else "하선"
    else:
        direction = "상선" if start_idx < end_idx else "하선"

    return start_idx, end_idx, direction

def print_congestion_predictions(start_station, end_station, input_YEAR, input_MONTH, input_DAY, input_LINE, input_TIME, station_data):
    filtered_station_data = [item for item in station_data if item[1] == input_LINE]
    stations = [item[2] for item in filtered_station_data]
    
    start_idx, end_idx, direction = determine_direction_and_indices(start_station, end_station, stations, input_LINE)
    
    model = joblib.load(f'XGB_reg_num{input_LINE}.pkl')

    congestion_values = []
    for idx in range(start_idx, end_idx + 1):
        station = stations[idx % len(stations)]
        input_data = pd.DataFrame({
            'YEAR': [input_YEAR],
            'MONTH': [input_MONTH],
            'DAY': [input_DAY],
            'STATION': [station],
            'DIRECTION': [direction],
            'TIME_00': [input_TIME]
        })

        prediction = model.predict(input_data)
        print(f"역: {station}, 예측 혼잡도: {prediction[0]:.2f}")
        congestion_values.append(prediction[0])

    avg_congestion = sum(congestion_values) / len(congestion_values)
    print(f"평균 혼잡도: {avg_congestion:.2f}")

In [82]:
print_congestion_predictions('서울역', '동대문', 2023, 5, '월', 1, 'TIME_18', list_data)

역: 서울역, 예측 혼잡도: 49.66
역: 시청, 예측 혼잡도: 65.14
역: 종각, 예측 혼잡도: 73.48
역: 종로3가, 예측 혼잡도: 78.90
역: 종로5가, 예측 혼잡도: 84.44
역: 동대문, 예측 혼잡도: 81.32
평균 혼잡도: 72.16


In [89]:
import pandas as pd
import joblib

df = pd.read_csv("호선정리.csv", encoding='CP949')
list_data = df.values.tolist()

def determine_direction_and_indices(start_station, end_station, stations, line_num):
    start_idx = stations.index(start_station)
    end_idx = stations.index(end_station)
    
    if line_num == 2 and abs(end_idx - start_idx) > len(stations) // 2:
        if start_idx < end_idx:
            direction = "하선"
            start_idx, end_idx = end_idx, start_idx + len(stations)
        else:
            direction = "상선"
            end_idx += len(stations)
    else:
        direction = "상선" if start_idx < end_idx else "하선"

    return start_idx, end_idx, direction

def print_congestion_predictions(start_station, end_station, input_YEAR, input_MONTH, input_DAY, input_LINE, input_TIME, station_data):
    stations = [item[2] for item in station_data if item[1] == input_LINE]
    start_idx, end_idx, direction = determine_direction_and_indices(start_station, end_station, stations, input_LINE)
    
    model = joblib.load(f'XGB_reg_num{input_LINE}.pkl')
    congestion_values = []

    for idx in range(start_idx, end_idx + 1):
        station = stations[idx % len(stations)]
        input_data = pd.DataFrame({
            'YEAR': [input_YEAR],
            'MONTH': [input_MONTH],
            'DAY': [input_DAY],
            'STATION': [station],
            'DIRECTION': [direction],
            'TIME_00': [input_TIME]
        })

        prediction = model.predict(input_data)
        print(f"역: {station}, 예측 혼잡도: {prediction[0]:.2f}")
        congestion_values.append(prediction[0])

    avg_congestion = sum(congestion_values) / len(congestion_values)
    print(f"평균 혼잡도: {avg_congestion:.2f}")

In [91]:
print_congestion_predictions('시청', '충정로', 2023, 5, '월', 2, 'TIME_18', list_data)

역: 충정로, 예측 혼잡도: 107.72
역: 용답, 예측 혼잡도: 43.01
역: 신답, 예측 혼잡도: 37.44
역: 신설동, 예측 혼잡도: 3.75
역: 도림천, 예측 혼잡도: 55.26
역: 양천구청, 예측 혼잡도: 45.21
역: 신정네거리, 예측 혼잡도: 39.39
역: 용두, 예측 혼잡도: 31.76
역: 까치산, 예측 혼잡도: 33.55
역: 시청, 예측 혼잡도: 103.45
평균 혼잡도: 50.05
